
## **Evaluación II** ##

### *Evaluación de contenidos 2 Módulo 2* ###

In [147]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import mysql.connector


Como en evaluaciones anteriores dispondréis de 2 tardes (aproximadamente 8 horas para la resolución de todos los ejercicios propuestos).
Estos ejercicios se tendrán que defender en el día de la evaluación de forma individual con vuestra tutora docente en una simulación de entrevista técnica donde tendréis de 20 minutos para resolver o explicar los ejercicios propuestos entregados.

En esta evaluación nos enfrentamos a un desafío emocionante de trabajar en un proyecto real para una empresa que realiza un estudio de universidades por el mundo. El proyecto tiene como objetivo identificar todas las universidades ubicadas en tres países específicos: Estados Unidos, Canadá y Argentina.

Para llevar a cabo esta tarea, utilizaremos la API de "Universities Hipolabs", una fuente confiable y completa de información sobre las universidades en todo el mundo. Con la ayuda de esta API, podemos acceder a una gran cantidad de datos relevantes, incluyendo el nombre de la universidad, la ciudad donde esta ubicada, el nombre de la institución y otra información importante que nos permitirá llevar a cabo un análisis detallado.

Es importante tener en cuenta que este proyecto requerirá un conocimiento profundo de herramientas y técnicas de análisis de datos, así como habilidades en programación y manejo de APIs. También es importante tener una comprensión sólida de la estructura y organización de los datos, ya que esto nos permitirá hacer preguntas importantes y obtener respuestas significativas a partir de los datos.

En resumen, esta prueba técnica ofrece una excelente oportunidad para demostrar habilidades y conocimientos en análisis de datos y programación, mientras se trabaja en un proyecto real y relevante para una empresa. Al finalizar del proyecto, esperamos obtener información valiosa que ayudará a la empresa a tomar decisiones más informadas sobre las universidades en los tres países objetivo.

1. Utilizando la API extraed toda la información que podáis de ella. La url para hacer las llamadas es:


In [148]:
API_URL = "http://universities.hipolabs.com/search?country=NOMBREPAIS"

In [149]:


url= "http://universities.hipolabs.com/search?country=canada"
response = requests.get(url)
frase_json = response.json()
json_text = response.text
json_structure = response.json()


In [150]:
print(json_structure)

[{'state-province': 'Quebec', 'domains': ['cstj.qc.ca'], 'name': 'Cégep de Saint-Jérôme', 'country': 'Canada', 'web_pages': ['https://www.cstj.qc.ca', 'https://ccmt.cstj.qc.ca', 'https://ccml.cstj.qc.ca'], 'alpha_two_code': 'CA'}, {'state-province': 'Ontario', 'domains': ['lambtoncollege.ca', 'mylambton.ca'], 'name': 'Lambton College', 'country': 'Canada', 'web_pages': ['https://www.lambtoncollege.ca'], 'alpha_two_code': 'CA'}, {'state-province': 'Nova Scotia', 'domains': ['acadiau.ca'], 'name': 'Acadia University', 'country': 'Canada', 'web_pages': ['http://www.acadiau.ca/'], 'alpha_two_code': 'CA'}, {'state-province': 'Ontario', 'domains': ['algonquincollege.com'], 'name': 'Algonquin College', 'country': 'Canada', 'web_pages': ['http://www.algonquincollege.com/'], 'alpha_two_code': 'CA'}, {'state-province': 'British Columbia', 'domains': ['ashtoncollege.com'], 'name': 'Ashton College', 'country': 'Canada', 'web_pages': ['http://www.ashtoncollege.com/'], 'alpha_two_code': 'CA'}, {'sta

### Definimos las variables ###

In [151]:
contry_name1 = "Argentina"
contry_name2 = "Canada"
contry_name3 = "United States"

### Creamos la classe ###

In [152]:

def API_call(contry_name):

    url = f'http://universities.hipolabs.com/search?country={contry_name}'

    response = requests.get(url)
    status_code = response.status_code
    status_reason = response.reason
    if  status_code == 200:
        print('The request was succeful. Status code:', status_code,'. Reason:',status_reason)
    elif  status_code == 402:
        print('Non-authorized user. Status code:',  status_code,'. Reason:',status_reason)
    elif  status_code == 404:
        print('Information not found. Status code:',  status_code,'. Reason:',status_reason)
    else:
        print('Unexpected error. Status code:',  status_code,'. Reason:',status_reason)

    df = pd.DataFrame.from_dict(pd.json_normalize(response.json()))
    
    return df

def unify_df(df):
    df_unified = pd.concat([df_argentina, df_canada, df_united_states], axis = 0)
    return df_unified





In [153]:
df_argentina = API_call(contry_name1)

The request was succeful. Status code: 200 . Reason: OK


In [154]:
df_canada = API_call(contry_name2)

The request was succeful. Status code: 200 . Reason: OK


In [155]:
df_united_states = API_call(contry_name3)

The request was succeful. Status code: 200 . Reason: OK


In [156]:
df_unified = pd.concat([df_argentina, df_canada, df_united_states], axis = 0)

df_unified

,state-province,domains,name,country,web_pages,alpha_two_code
0,Buenos Aires,[atlantida.edu.ar],Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR
1,Buenos Aires,[austral.edu.ar],Universidad Austral Buenos Aires,Argentina,[http://www.austral.edu.ar/],AR
2,Ciudad Autónoma de Buenos Aires,[caece.edu.ar],"Universidad CAECE, Buenos Aires",Argentina,[http://www.caece.edu.ar/],AR
3,Ciudad Autónoma de Buenos Aires,[cema.edu.ar],Instituto Universitario CEMA,Argentina,[http://www.cema.edu.ar/],AR
4,Ciudad Autónoma de Buenos Aires,[iese.edu.ar],Instituto de Enseñanza Superior del Ejército,Argentina,[http://www.iese.edu.ar/],AR
...,...,...,...,...,...,...
2276,None,[vul.edu],Virginia University of Lynchburg,United States,[https://www.vul.edu/],US
2277,None,[voorhees.edu],Voorhees University,United States,[https://www.voorhees.edu/],US
2278,None,[wvstate.edu],West Virginia State University,United States,[https://www.wvstateu.edu/],US
2279,None,[wileyc.edu],Wiley College,United States,[https://www.wileyc.edu/],US


In [157]:
df_unified['country'].unique()

array(['Argentina', 'Canada', 'United States'], dtype=object)

2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:
Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.
La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.



In [158]:
def clean(df):
    

    
    new_columns = {col: col.replace("-", "_") for col in df_unified.columns}
    df_unified.rename(columns = new_columns, inplace = True)
   
    df_unified.drop(['domains'], axis=1, inplace=True)
    return df_unified
    

In [159]:
clean(df_unified)

,state_province,name,country,web_pages,alpha_two_code
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR
1,Buenos Aires,Universidad Austral Buenos Aires,Argentina,[http://www.austral.edu.ar/],AR
2,Ciudad Autónoma de Buenos Aires,"Universidad CAECE, Buenos Aires",Argentina,[http://www.caece.edu.ar/],AR
3,Ciudad Autónoma de Buenos Aires,Instituto Universitario CEMA,Argentina,[http://www.cema.edu.ar/],AR
4,Ciudad Autónoma de Buenos Aires,Instituto de Enseñanza Superior del Ejército,Argentina,[http://www.iese.edu.ar/],AR
...,...,...,...,...,...
2276,None,Virginia University of Lynchburg,United States,[https://www.vul.edu/],US
2277,None,Voorhees University,United States,[https://www.voorhees.edu/],US
2278,None,West Virginia State University,United States,[https://www.wvstateu.edu/],US
2279,None,Wiley College,United States,[https://www.wileyc.edu/],US



3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejericio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva.  tenéis la documentación de este método.

In [160]:
df_unified = df_unified.explode('web_pages')

In [161]:


df_unified 

,state_province,name,country,web_pages,alpha_two_code
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,http://www.atlantida.edu.ar/,AR
1,Buenos Aires,Universidad Austral Buenos Aires,Argentina,http://www.austral.edu.ar/,AR
2,Ciudad Autónoma de Buenos Aires,"Universidad CAECE, Buenos Aires",Argentina,http://www.caece.edu.ar/,AR
3,Ciudad Autónoma de Buenos Aires,Instituto Universitario CEMA,Argentina,http://www.cema.edu.ar/,AR
4,Ciudad Autónoma de Buenos Aires,Instituto de Enseñanza Superior del Ejército,Argentina,http://www.iese.edu.ar/,AR
...,...,...,...,...,...
2276,None,Virginia University of Lynchburg,United States,https://www.vul.edu/,US
2277,None,Voorhees University,United States,https://www.voorhees.edu/,US
2278,None,West Virginia State University,United States,https://www.wvstateu.edu/,US
2279,None,Wiley College,United States,https://www.wileyc.edu/,US


4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.
   


In [162]:
df_unified.duplicated('name').sum()

df_unified = df_unified.drop_duplicates(subset= 'name')

df_unified

,state_province,name,country,web_pages,alpha_two_code
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,http://www.atlantida.edu.ar/,AR
1,Buenos Aires,Universidad Austral Buenos Aires,Argentina,http://www.austral.edu.ar/,AR
2,Ciudad Autónoma de Buenos Aires,"Universidad CAECE, Buenos Aires",Argentina,http://www.caece.edu.ar/,AR
3,Ciudad Autónoma de Buenos Aires,Instituto Universitario CEMA,Argentina,http://www.cema.edu.ar/,AR
4,Ciudad Autónoma de Buenos Aires,Instituto de Enseñanza Superior del Ejército,Argentina,http://www.iese.edu.ar/,AR
...,...,...,...,...,...
2276,None,Virginia University of Lynchburg,United States,https://www.vul.edu/,US
2277,None,Voorhees University,United States,https://www.voorhees.edu/,US
2278,None,West Virginia State University,United States,https://www.wvstateu.edu/,US
2279,None,Wiley College,United States,https://www.wileyc.edu/,US


   
5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.
   


In [163]:
def replace_none_with_nan(df, column):
    return df_unified[column].fillna(value=pd.np.nan)

In [164]:
df_unified['state_province'] = replace_none_with_nan(df_unified, 'state_province')

/var/folders/qs/lczv8kqx5w59ygqwfs1w0vlr0000gn/T/ipykernel_6920/1580227518.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  return df_unified[column].fillna(value=pd.np.nan)
/var/folders/qs/lczv8kqx5w59ygqwfs1w0vlr0000gn/T/ipykernel_6920/2188881352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unified['state_province'] = replace_none_with_nan(df_unified, 'state_province')


In [166]:
df_unified

,state_province,name,country,web_pages,alpha_two_code
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,http://www.atlantida.edu.ar/,AR
1,Buenos Aires,Universidad Austral Buenos Aires,Argentina,http://www.austral.edu.ar/,AR
2,Ciudad Autónoma de Buenos Aires,"Universidad CAECE, Buenos Aires",Argentina,http://www.caece.edu.ar/,AR
3,Ciudad Autónoma de Buenos Aires,Instituto Universitario CEMA,Argentina,http://www.cema.edu.ar/,AR
4,Ciudad Autónoma de Buenos Aires,Instituto de Enseñanza Superior del Ejército,Argentina,http://www.iese.edu.ar/,AR
...,...,...,...,...,...
2276,NaN,Virginia University of Lynchburg,United States,https://www.vul.edu/,US
2277,NaN,Voorhees University,United States,https://www.voorhees.edu/,US
2278,NaN,West Virginia State University,United States,https://www.wvstateu.edu/,US
2279,NaN,Wiley College,United States,https://www.wileyc.edu/,US


6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.
   


In [169]:
df_unified['state_province'].fillna(value= 'unknown', inplace= True)

/var/folders/qs/lczv8kqx5w59ygqwfs1w0vlr0000gn/T/ipykernel_6920/3893195183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unified['state_province'].fillna(value= 'Unknown', inplace= True)


In [170]:
df_unified

,state_province,name,country,web_pages,alpha_two_code
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,http://www.atlantida.edu.ar/,AR
1,Buenos Aires,Universidad Austral Buenos Aires,Argentina,http://www.austral.edu.ar/,AR
2,Ciudad Autónoma de Buenos Aires,"Universidad CAECE, Buenos Aires",Argentina,http://www.caece.edu.ar/,AR
3,Ciudad Autónoma de Buenos Aires,Instituto Universitario CEMA,Argentina,http://www.cema.edu.ar/,AR
4,Ciudad Autónoma de Buenos Aires,Instituto de Enseñanza Superior del Ejército,Argentina,http://www.iese.edu.ar/,AR
...,...,...,...,...,...
2276,unknown,Virginia University of Lynchburg,United States,https://www.vul.edu/,US
2277,unknown,Voorhees University,United States,https://www.voorhees.edu/,US
2278,unknown,West Virginia State University,United States,https://www.wvstateu.edu/,US
2279,unknown,Wiley College,United States,https://www.wileyc.edu/,US


   
7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso,  la documentación. Para desarrollar este ejercicio deberéis:
Sacar los valores únicos de la columna state_province.

Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:

NV: reemplazalo por Nevada
TX: reemplazalo por Texas
IN: reemplazalo por Indianapolis
CA: reemplazalo por California
VA: reemplazalo por Virginia
NY: reemplazalo por New York
MI: reemplazalo por Michigan
GA: reemplazalo por Georgia
ND: reemplazalo por North Dakota

Otros valores que tenemos más formateados son y que deberemos reemplazar:
New York, NY. Deberéis reemplazarlo por "New York".

'Buenos Aires', 'Ciudad Autónoma de Buenos Aires'. En este caso deberéis poner en ambos casos "Buenos Aires"

Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.

Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.
Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.


In [174]:
df_unified['state_province'].unique()

array(['Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'unknown',
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado', 'IN', 'CA',
       'South Carolina', 'Washington', 'NY', 'Texas', 'ND', 'MI', 'Ohio',
       'Florida', 'California', 'North Carolina', 'Michigan', 'GA',
       'New York, NY'], dtype=object)

In [179]:
dict_provinces = {"NV":"Nevada", "TX":"Texas", "IN": "Indianapolis", "CA":"California", "VA": "Virgina", 
                  "NY": "New York", "MI":"Michigan", "GA": "Georgia", "ND":"North Dakota", "Ciudad Autónoma de Buenos Aires": "Buenos Aires"}

In [180]:
df_unified['state_province'].replace(dict_provinces, inplace= True)

/var/folders/qs/lczv8kqx5w59ygqwfs1w0vlr0000gn/T/ipykernel_6920/2324965444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unified['state_province'].replace(dict_provinces, inplace= True)


In [181]:
df_unified['state_province'].unique()

array(['Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza',
       'Santa Fé', 'unknown', 'Santiago Del Estero', 'Misiones',
       'Catamarca', 'Formosa', 'Jujuy', 'La Rioja', 'La Pampa',
       'San Juan', 'San Luis', 'Tucumán', 'Quebec', 'Ontario',
       'Nova Scotia', 'British Columbia', 'Alberta', 'Manitoba',
       'New Brunswick', 'Saskatchewan', 'Newfoundland and Labrador',
       'Prince Edward Island', 'Yukon', 'Pennsylvania', 'Nevada', 'Iowa',
       'Virgina', 'Texas', 'Colorado', 'Indianapolis', 'California',
       'South Carolina', 'Washington', 'New York', 'North Dakota',
       'Michigan', 'Ohio', 'Florida', 'North Carolina', 'Georgia',
       'New York, NY'], dtype=object)

In [182]:
provinces_list = ['Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza',
       'Santa Fé', 'unknown', 'Santiago Del Estero', 'Misiones',
       'Catamarca', 'Formosa', 'Jujuy', 'La Rioja', 'La Pampa',
       'San Juan', 'San Luis', 'Tucumán', 'Quebec', 'Ontario',
       'Nova Scotia', 'British Columbia', 'Alberta', 'Manitoba',
       'New Brunswick', 'Saskatchewan', 'Newfoundland and Labrador',
       'Prince Edward Island', 'Yukon', 'Pennsylvania', 'Nevada', 'Iowa',
       'Virgina', 'Texas', 'Colorado', 'Indianapolis', 'California',
       'South Carolina', 'Washington', 'New York', 'North Dakota',
       'Michigan', 'Ohio', 'Florida', 'North Carolina', 'Georgia',
       'New York, NY']

In [177]:
df_unified

,state_province,name,country,web_pages,alpha_two_code
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,http://www.atlantida.edu.ar/,AR
1,Buenos Aires,Universidad Austral Buenos Aires,Argentina,http://www.austral.edu.ar/,AR
2,Ciudad Autónoma de Buenos Aires,"Universidad CAECE, Buenos Aires",Argentina,http://www.caece.edu.ar/,AR
3,Ciudad Autónoma de Buenos Aires,Instituto Universitario CEMA,Argentina,http://www.cema.edu.ar/,AR
4,Ciudad Autónoma de Buenos Aires,Instituto de Enseñanza Superior del Ejército,Argentina,http://www.iese.edu.ar/,AR
...,...,...,...,...,...
2276,unknown,Virginia University of Lynchburg,United States,https://www.vul.edu/,US
2277,unknown,Voorhees University,United States,https://www.voorhees.edu/,US
2278,unknown,West Virginia State University,United States,https://www.wvstateu.edu/,US
2279,unknown,Wiley College,United States,https://www.wileyc.edu/,US



8. Crea una BBDD en mysql que contenga las siguientes tablas:
   
- Tabla países: donde encontraremos las siguientes columnas:
  
  - idestado: primary key, integer, autoincremental
  - nombre_pais: varchar
  - nombre_provincia: varchar
  - latitud: decimal
  - longitud: decimal
  -----------------

- Tabla universidades: donde encontraremos las siguientes columnas:
  
  - iduniversidades: primary key, integer, autoincremental
  - nombre_universidad: varchar
  - pagina_web: varchar
  - paises_idestado: foreing key

9. Introduce todo el código que habéis ido creando en funciones, siguiendo la misma lógica que hemos seguido en los pairs

10. BONUS

- Introduce los datos en la BBDD de SQL.
- Crea una clase con todo el código generado en esta evaluación.
